In [289]:
#Graph imports
import re
import pydot

In [290]:
#GUI imports
import io
import _tkinter
import tkinter as tk
from tkinter import * 
from tkinter import ttk
from tkinter import filedialog
from PIL import Image, ImageTk
from IPython.core.display import Image as image

In [291]:
def get_risk_graph(file_name):
    G = pydot.Dot(graph_type='graph')    
    #Assumption max of 6 Partitions
    partitions_colors = ['red','yellow','green','blue','violet','pink']
    nodes_bonus = {}
    file = open(file_name, "r")
    line_1 = file.readline()
    line_1 = re.split(' |\n',line_1)
    line_1.remove('')
    n_nodes = int(line_1[0])
    n_edges = int(line_1[1])
    n_partitions = int(line_1[2])
    start_index = 2
    for i in range(start_index, start_index + n_nodes):
        start_index = i+1
        line = file.readline(i)
        node_name = re.split(' |\n',line)
        node_name = node_name[0]
        node = pydot.Node(node_name, style='filled',shape='circle',fillcolor='pink')
        G.add_node(node)
    for i in range(start_index, start_index + n_edges):
        start_index = i+1
        line = file.readline(i)
        line = re.split(' |\n',line)        
        node1 = line[0]
        node2 = line[1]
        edge = pydot.Edge(node1, node2)
        G.add_edge(edge)
    for i in range(start_index, start_index + n_partitions):
        line = file.readline(i)
        line = re.split(' |\n',line)
        line.remove('')
        bonus_value = int(line[0])
        del line[0]        
        node_color = partitions_colors[i-start_index]
        for node_name in line:
            nodes_bonus[node_name] = bonus_value
            node = G.get_node(node_name)            
            node[0].set_fillcolor(node_color)           
    file.close()
    return (G,nodes_bonus)

In [292]:
#Initializing root frame 
root=tk.Tk()
root.title("RISK")

''

In [293]:
#Initializaing Global Variables Game
current_player = 1
## Globals of place frame
place_list = ["1","2","3","4","5","6","7","8","9","10","11","12"]
player_str = StringVar()
stage_str = StringVar()
countries_str = StringVar()
troops_str = StringVar()
bonus_str = StringVar()
place_str = StringVar()
control_frame = ttk.Frame(root)
attack_frame = ttk.Frame(root)
## Globals of attack frame
attack_list = ["1","2","3","4","5","6","7","8","9","10","11","12"]
attack_str = StringVar()

In [294]:
#Initializaing Global Variables
countries = 0 
troops = 0
bonus = 0

In [295]:
#Initializaing Global Variables Player 1
countries_1 = []
troops_1 = {}
bonus_1 = 0

In [296]:
#Initializaing Global Variables Player 2
countries_2 = []
troops_2 = {}
bonus_2 = 0

In [297]:
#Allowing expansion with scale
root.columnconfigure(0,weight=1)
root.columnconfigure(1,weight=1)
root.columnconfigure(2,weight=1)
root.columnconfigure(3,weight=1)
root.rowconfigure(0,weight=1)

In [298]:
#Load Game File
file_name = filedialog.askopenfilename(filetypes = (("Text files","*.txt"),("All files","*.*")))
G,nodes_bonus = get_risk_graph(file_name)

In [299]:
#Design the screen - graph frame
graph_frame = ttk.Frame(root)
graph_frame.config(width = 600, height = 800, relief = RIDGE)
graph_frame.grid(row=0,column=0,columnspan=2)

In [300]:
def build_ctrl_frame():
    global control_frame
    global current_player
    #Design the screen - control frame
    control_frame = ttk.Frame(root)
    control_frame.config(width = 300, height = 800)
    control_frame.grid(row=0,column=2,columnspan=2)
    #Components inside control frame - Placement mode 
    player_label = Label(control_frame, font="Times 18", textvariable = player_str)
    player_str.set("Player " + str(current_player))
    player_label.pack()
    stage_label = Label(control_frame,font="Times 18", textvariable = stage_str)
    stage_str.set("Troop Placement")
    stage_label.pack()
    countries_label = Label(control_frame,font="Times 14", textvariable = countries_str)
    countries_str.set("Countries = " + str(countries))
    countries_label.pack()
    troops_label = Label(control_frame,font="Times 14", textvariable = troops_str)
    troops_str.set("Troops = " + str(troops))
    troops_label.pack()
    bonus_label = Label(control_frame,font="Times 14", textvariable = bonus_str)
    bonus_str.set("Bonus = " + str(bonus))
    bonus_label.pack()
    choose_label = Label(control_frame, text="Choose Country to Place Troops", font="Times 12")
    choose_label.pack()
    place_str.set(place_list[0]) 
    place_menu = OptionMenu(control_frame, place_str, *place_list)
    place_menu.pack()
    place_button = tk.Button(control_frame, text="Place Troops",command=lambda: show_attack_frame())
    place_button.pack()

In [301]:
def build_attack_frame():
    global attack_frame
    global current_player
    #Design the screen - attack frame
    attack_frame = ttk.Frame(root)
    attack_frame.config(width = 300, height = 800)
    attack_frame.grid(row=0,column=2,columnspan=2)
    #Components inside control frame - Placement mode 
    player_label = Label(attack_frame, font="Times 18", textvariable = player_str)
    player_str.set("Player " + str(current_player))
    player_label.pack()
    stage_label = Label(attack_frame,font="Times 18", textvariable = stage_str)
    stage_str.set("Attacking")
    stage_label.pack()
    countries_label = Label(attack_frame,font="Times 14", textvariable = countries_str)
    countries_str.set("Countries = " + str(countries))
    countries_label.pack()
    troops_label = Label(attack_frame,font="Times 14", textvariable = troops_str)
    troops_str.set("Troops = " + str(troops))
    troops_label.pack()
    choose_label = Label(attack_frame, text="Choose Attacking country", font="Times 12")
    choose_label.pack()
    attack_str.set(attack_list[0]) 
    attack_menu = OptionMenu(attack_frame, attack_str, *attack_list)
    attack_menu.pack()
    attack_button = tk.Button(attack_frame, text="Attack",command=lambda: show_control())
    attack_button.pack()
    skip_button = tk.Button(attack_frame, text="Skip",command=lambda: show_control())
    skip_button.pack()

In [302]:
def show_control():
    global attack_frame
    attack_frame.grid_forget()
    build_ctrl_frame()    

In [303]:
def show_attack_frame():
    global control_frame
    control_frame.grid_forget()
    build_attack_frame()    

In [304]:
#Start Building Ctrl Frame
build_ctrl_frame()

In [305]:
#Show the main Graph of the Game
pilimg = Image.open(io.BytesIO(G.create_png()))
image_tk = ImageTk.PhotoImage(pilimg, master = graph_frame)
label = tk.Label(root, text = 'Input Image', image = image_tk, background = 'white')
label.image = image_tk
label.grid(row=0, column=0, sticky="snew", columnspan=2)

In [306]:
root.mainloop()